In [102]:
import plotly.graph_objects as go
import math
import numpy as np


In [ ]:
class Order():
    def __init__(self,price,volume,isBuy,owner = None):
        self.price = price
        self.volume = volume
        self.isBuy = isBuy
        self.unfilledVolume = volume
        self.executionPrices = []
        self.owner = owner
        self.location = None

    
    def __str__(self):
        if self.owner == None:
            return f"Owner={self.owner}, Price={self.price}, Volume={self.volume}, Unfilled={self.unfilledVolume}"
        else:
            return f"Owner={self.owner.name}, Price={self.price}, Volume={self.volume}, Unfilled={self.unfilledVolume}"
    
        
    def execute(self,execution): # execution is a tuple in form (amount,price)
        self.executionPrices.append(execution)
        self.unfilledVolume -= execution[0] 

        if self.owner != None:
            if self.isBuy:    
                self.owner.cash -= execution[0] * execution[1]
                self.owner.position += execution[0]
            else:
                self.owner.cash += execution[0] * execution[1]
                self.owner.position -= execution[0]

            if self.unfilledVolume == 0:
                self.location = None
                pass 



In [ ]:
class Node():
    def __init__(self,value,orders = None):
        self.value = value
        self.left = None
        self.right = None
        self.parent = None
        if orders == None: #for increased efficiency change orders to a linked list but I am to lazy to do that right now
            self.orders = []
        else:
            self.orders = orders

    
    def __str__(self): #does inorder traversal returning all of the orders
        output  = []
        for order in self.orders:
            output.append(str(order))
        return ".\n".join(output)
    

    def in_order(self):
        if self.left != None:
            yield from self.left.in_order()

        yield self.orders,self
        
        if self.right != None:
            yield from self.right.in_order()


    def insert(self,order,direction): 
        if order.price == self.value:
            self.orders.append(order) 
            order.location = self
    
        elif order.price * direction < self.value * direction: #move to right node
            if self.right == None:
                self.right = Node(order.price,[order])
                self.right.parent = self
                order.location = self.right
            else:
                self.right.insert(order,direction)

        else: #move to left node
            if self.left == None:
                self.left = Node(order.price,[order])
                self.left.parent = self
                order.location = self.left
            else:
                self.left.insert(order,direction)

        

In [ ]:
class OrderTree():
    def __init__(self,isBuy,node = None):
        self.head = node
        self.isBuy = isBuy

    def bestPrice(self):
        if self.head == None:
            if self.isBuy:
                return -math.inf
            else:
                return math.inf
        else:
            currentNode = self.head
            while currentNode.left is not None:
                currentNode = currentNode.left
            return currentNode.value

    
    def __str__(self):
        header = f"bestPrice={self.bestPrice()}"
        if self.head == None:
            tree = "empty"
        else:
            tree = ".\n".join([str(order) for orders,_ in self.head.in_order() for order in orders])
        return header + '\n' + tree
    

    def insertOrder(self,order): 
        if self.head is None: #to optimize use a self balancing tree
            node = Node(order.price, [order])
            self.head = node
            order.location = node
        else:
            direction = 1 if self.isBuy else -1
            self.head.insert(order, direction)

    def transplant(self,u, v):
        if u.parent is None:
            self.head = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        if v:
            v.parent = u.parent


    def removeNode(self, node):
        # Case 1 & 2: node has less than 2 children
        if node.left is None:
            self.transplant(node, node.right)
        elif node.right is None:
            self.transplant(node, node.left)
        else:
            # Find in-order predecessor (max of left subtree)
            pred = node.left
            while pred.right:
                pred = pred.right

            # If pred is not node.left, move pred up
            if pred.parent != node:
                self.transplant(pred, pred.left)
                pred.left = node.left
                if pred.left:
                    pred.left.parent = pred

            self.transplant(node, pred)
            pred.right = node.right
            if pred.right:
                pred.right.parent = pred





In [ ]:
class orderbook:
    def __init__(self):
        self.sells = OrderTree(False)
        self.buys = OrderTree(True)


    def __str__(self):
        return "---------------\nBuy Book \n---------------\n" + str(self.buys) + "\n" + "---------------\nSell Book \n---------------\n" + str(self.sells) 

    def submitOrder(self,order): #ensure that it checks that the price is still currently a valid price

        deleteNodes = []
        
        if order.isBuy: #submitting a buy order
            if order.price < self.sells.bestPrice(): #no order fill
                self.buys.insertOrder(order)
            else: #order fillable
                for existingOrders, currentNode in self.sells.head.in_order(): # fills order on root node

                    if order.price < currentNode.value:
                        break
                    updatedOrders = []

                    for existingOrder in existingOrders:
                        if existingOrder.unfilledVolume <= order.unfilledVolume:
                            #fill entire order
                            order.execute((existingOrder.unfilledVolume,existingOrder.price))
                            existingOrder.execute((existingOrder.unfilledVolume,existingOrder.price))
                        else:
                            #partial fill the orde
                            existingOrder.execute((order.unfilledVolume,existingOrder.price)) #adds tuple of trade execution to the order and sets remaning volume to zero
                            order.execute((order.unfilledVolume,existingOrder.price))
                            updatedOrders.append(existingOrder)
                            break
                        if existingOrder.unfilledVolume > 0:
                            updatedOrders.append(existingOrder)

                    currentNode.orders = updatedOrders
                    if len(updatedOrders) == 0:
                        deleteNodes.append(currentNode)


                    if order.unfilledVolume <= 0:#order is completely filled
                        break
                if order.unfilledVolume > 0:#insert remaining unfilled volume onto the market
                    self.buys.insertOrder(order)

                for node in deleteNodes:
                    if node.parent == None: 
                        self.sells.head = self.sells.head.right
                        if self.sells.head != None:
                            self.sells.head.parent = None
                    else:
                        if node.right == None:
                            node.parent.left = node.right
                        else:
                            node.right.parent = node.parent
                            node.parent.left = node.right
                        



        else: #submitting a sell order
            if order.price > self.buys.bestPrice(): #no order fill
                self.sells.insertOrder(order)
            else: #order fillable
                for existingOrders, currentNode in self.buys.head.in_order(): # fills order on root node

                    if order.price > currentNode.value:
                        break
                    updatedOrders = []

                    for existingOrder in existingOrders:
                        if existingOrder.unfilledVolume <= order.unfilledVolume:
                            #fill entire order
                            order.execute((existingOrder.unfilledVolume,existingOrder.price))
                            existingOrder.execute((existingOrder.unfilledVolume,existingOrder.price))
                        else:
                            #partial fill the order
                            existingOrder.execute((order.unfilledVolume,existingOrder.price)) #adds tuple of trade execution to the order and sets remaning volume to zero
                            order.execute((order.unfilledVolume,existingOrder.price))
                            updatedOrders.append(existingOrder)
                            break
                        if existingOrder.unfilledVolume > 0:
                            updatedOrders.append(existingOrder)

                    currentNode.orders = updatedOrders
                    if len(updatedOrders) == 0:
                        deleteNodes.append(currentNode)


                    if order.unfilledVolume <= 0:#order is completely filled
                        break

                if order.unfilledVolume > 0: #insert remaining unfilled volume onto the market
                    self.sells.insertOrder(order)
        
                for node in deleteNodes:
                    if node.parent == None: 
                        self.buys.head = self.buys.head.right
                        if self.buys.head != None:
                            self.buys.head.parent = None
                    else:
                        if node.right == None:
                            node.parent.left = node.right
                        else:
                            node.right.parent = node.parent
                            node.parent.left = node.right


        
    def cancelOrder(self,order):
        node = order.location
        if node != None: #if the location is none the order is unplaced.
            if order in node.orders:
                node.orders.remove(order)
                if len(node.orders) == 0:   
                    if order.isBuy:
                        self.buys.removeNode(node)
                    else:
                        self.sells.removeNode(node)

            order.location = None


In [107]:
class member:
    def __init__(self,name,cash = 0):
        self.name = name
        self.cash = cash
        self.position = 0
        self.sellOrders = []
        self.buyOrders = []

    def __str__(self):
        return f"Name={self.name}, Cash={self.cash}, Position={self.position}"
    def submitOrder(self,exchange,order):
        order.owner = self
        if order.isBuy:
            self.buyOrders.append(order)
        else:
            self.sellOrders.append(order)
        
        exchange.submitOrder(order)

    def removeOrder(self,exchange,order):
        exchange.cancelOrder(order)

        if order.isBuy:
            self.buyOrders.remove(order)
        else:
            self.sellOrders.remove(order)

In [ ]:
import random
exchange = orderbook()
orderList = []


for x in range(200):
    
    order1 = Order(np.floor(np.random.normal()* 25) + 500,random.randint(1,500),False)
    order2 = Order(np.floor(np.random.normal()* 25) + 495,random.randint(1,500),True)
    o = random.choice([order1,order2])
    exchange.submitOrder(o)
    orderList.append(o)


    

'''
for x in range(1000):
    try:
        mid_price = int((exchange.sells.bestPrice() + exchange.buys.bestPrice())/2)
    except:
        mid_price = 500

    
    order1 = Order(np.floor(np.random.normal()* 25) + mid_price+2,random.randint(1,500),False)
    order2 = Order(np.floor(np.random.normal()* 25) + mid_price-2,random.randint(1,500),True)
    exchange.submitOrder(random.choice([order1,order2]))
'''

print(exchange)

---------------
Buy Book 
---------------
bestPrice=487.0
Owner=None, Price=487.0, Volume=496, Unfilled=415.
Owner=None, Price=486.0, Volume=429, Unfilled=311.
Owner=None, Price=484.0, Volume=320, Unfilled=320.
Owner=None, Price=483.0, Volume=126, Unfilled=126.
Owner=None, Price=483.0, Volume=454, Unfilled=454.
Owner=None, Price=481.0, Volume=87, Unfilled=87.
Owner=None, Price=481.0, Volume=479, Unfilled=479.
Owner=None, Price=481.0, Volume=87, Unfilled=87.
Owner=None, Price=480.0, Volume=242, Unfilled=242.
Owner=None, Price=479.0, Volume=488, Unfilled=488.
Owner=None, Price=479.0, Volume=110, Unfilled=110.
Owner=None, Price=478.0, Volume=286, Unfilled=286.
Owner=None, Price=478.0, Volume=428, Unfilled=428.
Owner=None, Price=478.0, Volume=231, Unfilled=231.
Owner=None, Price=478.0, Volume=113, Unfilled=113.
Owner=None, Price=478.0, Volume=60, Unfilled=60.
Owner=None, Price=477.0, Volume=126, Unfilled=126.
Owner=None, Price=477.0, Volume=177, Unfilled=177.
Owner=None, Price=476.0, Volum

In [109]:
MrMoney = member("moneyPants",0)

def updateExchange(exchange):
    for _ in range(random.randint(0,10)):
        try:
            mid_price = int((exchange.sells.bestPrice() + exchange.buys.bestPrice())/2)
        except:
            mid_price = 500


        
        order1 = Order(np.floor(np.random.normal()* 3) + mid_price+2,random.randint(1,500),False)
        order2 = Order(np.floor(np.random.normal()* 3) + mid_price-2,random.randint(1,500),True)
        o = random.choice([order1,order2])
        exchange.submitOrder(o)



        orderList.append(o)

        if random.randint(0,100) < 98:

            oc = random.choice(orderList)
            exchange.cancelOrder(oc)
            orderList.remove(oc)

    

    #market maker

    if len(MrMoney.sellOrders) > 0:


        if next(exchange.sells.head.in_order()) != MrMoney.sellOrders[0] or MrMoney.sellOrders[0].unfilledVolume == 0:
            MrMoney.removeOrder(exchange,MrMoney.sellOrders[0])
            MrMoney.submitOrder(exchange,Order(exchange.sells.bestPrice()-1,50,False))

    else:
        MrMoney.submitOrder(exchange,Order(exchange.sells.bestPrice()-1,50,False))



    if len(MrMoney.buyOrders) > 0:

        if next(exchange.buys.head.in_order()) != MrMoney.buyOrders[0] or MrMoney.buyOrders[0].unfilledVolume == 0:
            MrMoney.removeOrder(exchange,MrMoney.buyOrders[0])
            MrMoney.submitOrder(exchange,Order(exchange.buys.bestPrice()+1,50,True))

    else:
        MrMoney.submitOrder(exchange,Order(exchange.buys.bestPrice()+1,50,True))

    

    return exchange

In [115]:
#display exchange

# Buy side (sorted high to low)


buy_prices = []
buy_volumes = []

for orders, currentNode in exchange.buys.head.in_order():
    buy_prices.append(currentNode.value)
    buy_volumes.append(sum(order.unfilledVolume for order in orders))

# Sell side (sorted low to high)


sell_prices = []
sell_volumes = []

for orders, currentNode in exchange.sells.head.in_order():
    sell_prices.append(currentNode.value)
    sell_volumes.append(sum(order.unfilledVolume for order in orders))


buy_cumvol = np.cumsum(buy_volumes) 
sell_cumvol = np.cumsum(sell_volumes)           

fig = go.Figure()

# Buy side (bids)
fig.add_trace(go.Scatter(
    x=buy_prices,
    y=buy_cumvol,
    mode='lines+markers',
    name='Bids (Buy)',
    line=dict(color='green', shape='hv')
))

# Sell side (asks)
fig.add_trace(go.Scatter(
    x=sell_prices,
    y=sell_cumvol,
    mode='lines+markers',
    name='Asks (Sell)',
    line=dict(color='red', shape='hv')
))

fig.update_layout(
    title='Order Book Depth Chart',
    xaxis_title='Price',
    yaxis_title='Cumulative Volume',
    xaxis=dict(type='linear'),
    yaxis=dict(type='linear'),
    template='plotly_white'
)

fig.show()


In [114]:
import plotly.graph_objects as go
import numpy as np


frames = []
initial_bids = []
initial_asks = []

for t in range(500):
    buy_prices = []
    buy_volumes = []
    for orders, currentNode in exchange.buys.head.in_order():
        buy_prices.append(currentNode.value)
        buy_volumes.append(sum(order.unfilledVolume for order in orders))
    
    sell_prices = []
    sell_volumes = []
    for orders, currentNode in exchange.sells.head.in_order():
        sell_prices.append(currentNode.value)
        sell_volumes.append(sum(order.unfilledVolume for order in orders))

    buy_cumvol = np.cumsum(buy_volumes)
    sell_cumvol = np.cumsum(sell_volumes)

    # Save the first snapshot to use as the base
    if t == 0:
        initial_bids = [go.Scatter(
            x=buy_prices,
            y=buy_cumvol,
            mode='lines+markers',
            name='Bids (Buy)',
            line=dict(color='green', shape='hv')
        )]
        initial_asks = [go.Scatter(
            x=sell_prices,
            y=sell_cumvol,
            mode='lines+markers',
            name='Asks (Sell)',
            line=dict(color='red', shape='hv')
        )]

    # Each frame stores updated bid/ask lines
    frames.append(go.Frame(
        name=str(t),
        data=[
            go.Scatter(x=buy_prices, y=buy_cumvol, mode='lines+markers',
                       line=dict(color='green', shape='hv')),
            go.Scatter(x=sell_prices, y=sell_cumvol, mode='lines+markers',
                       line=dict(color='red', shape='hv'))
        ]
    ))

    exchange = updateExchange(exchange)

# Create figure with initial data
fig = go.Figure(
    data=initial_bids + initial_asks,
    layout=go.Layout(
        title="Order Book Depth Chart Over Time",
        xaxis_title="Price",
        yaxis_title="Cumulative Volume",
        template='plotly_white',
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play", method="animate", args=[None])]
        )]
    ),
    frames=frames
)

# Add slider for time
fig.update_layout(
    sliders=[{
        "steps": [
            {
                "method": "animate",
                "args": [[str(t)], {"mode": "immediate", "frame": {"duration": 200, "redraw": True}}],
                "label": f"t={t}"
            } for t in range(len(frames))
        ],
        "transition": {"duration": 0},
        "x": 0.1,
        "len": 0.9
    }]
)

fig.show()

In [112]:
print(MrMoney)
print(MrMoney.sellOrders)
print(MrMoney.buyOrders)

Name=moneyPants, Cash=-411905.0, Position=891


In [113]:
print(exchange)
exchange = updateExchange(exchange)

---------------
Buy Book 
---------------
bestPrice=390.0
Owner=None, Price=390.0, Volume=413, Unfilled=15.
Owner=None, Price=389.0, Volume=381, Unfilled=160.
Owner=None, Price=389.0, Volume=352, Unfilled=352.
Owner=None, Price=389.0, Volume=131, Unfilled=131.
Owner=None, Price=389.0, Volume=264, Unfilled=264.
Owner=None, Price=389.0, Volume=268, Unfilled=268.
Owner=None, Price=389.0, Volume=363, Unfilled=363.
Owner=None, Price=388.0, Volume=263, Unfilled=263.
Owner=None, Price=388.0, Volume=176, Unfilled=176.
Owner=None, Price=388.0, Volume=493, Unfilled=493.
Owner=None, Price=388.0, Volume=221, Unfilled=221.
Owner=None, Price=388.0, Volume=473, Unfilled=473.
Owner=None, Price=388.0, Volume=445, Unfilled=445.
Owner=None, Price=388.0, Volume=369, Unfilled=369.
Owner=None, Price=387.0, Volume=197, Unfilled=197.
Owner=None, Price=387.0, Volume=172, Unfilled=172.
Owner=None, Price=387.0, Volume=199, Unfilled=199.
Owner=None, Price=386.0, Volume=497, Unfilled=497.
Owner=None, Price=386.0, 